In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import load_pkl, GRAY_DASH, save_latex

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: "$\Delta$",
    IssueEnum.NEW: "#",
}

KINDS = {
    DepKind.STRUCT: "Struct",
    DepKind.CONFIG: "Config",
    DepKind.FUNC: "Func",
    DepKind.TRACEPOINT: r"\shortstack{Tracept\\($\Delta=0$)}",
    DepKind.SYSCALL: r"Syscall",
}


results = {}
for (g, p), d in data.items():
    if p.v2.flavor == "oracle":
        continue
    result = {}
    old = 0
    for (kind, s), v in d.items():
        if s == IssueEnum.OLD:
            old = v
        if kind == DepKind.CONFIG and s != IssueEnum.NEW:
            continue
        kind = KINDS[kind]
        s = ISSUES.get(s)
        if not s:
            continue
        if v > 1000:
            v = f"{v/1000:.1f}k"
        # result[(kind, s)] = f"{v / old * 100:.1f}"
        result[(kind, s)] = v
    result[("Register", ISSUES[IssueEnum.CHANGE])] = 1 if g == VersionGroup.ARCH else 0
    name = g.to_str(p.v2)
    if g == VersionGroup.ARCH:
        name = r"\texttt{\footnotesize " + name + r"}"
    results[(g, name)] = result


df = pd.DataFrame(results)
# df = df.loc[(df != 0).any(axis=1) | (df.index.get_level_values(1) == "$\Delta$")]
df = df.loc[(df != 0).any(axis=1)]


df_latex = df.map(lambda x: GRAY_DASH if x == 0 else x)
latex = df_latex.to_latex(
    multicolumn_format="c|", column_format="ll|rrrr|rrrr", multirow=True
)

save_latex(latex, "cfg", rotate=False)
df

[          pkl.py:18 ] INFO: Loding config from /Users/szhong/Downloads/bpf-study/output/config.pkl
[        latex.py:56 ] INFO: Saved cfg to /Users/szhong/Downloads/bpf-study/paper/tabs/cfg.tex


Arch                                                                                      Flavor                     
                                            \texttt{\footnotesize arm64} \texttt{\footnotesize arm32} \texttt{\footnotesize ppc} \texttt{\footnotesize s390} low-lat    AWS  Azure    GCP
Config                             #                                9.6k                         9.6k                       8.1k                        3.0k    8.8k   6.4k   5.3k   8.6k
Func                               #                               49.4k                        48.8k                      42.8k                       33.2k   48.0k  46.5k  45.5k  48.1k
                                   $+$                              9.2k                        12.6k                       5.4k                        3.9k      57    328    992    450
                                   $-$                              7.9k                        11.8k                      10.6k                       18.8k      41   1.8k   3.5k    319
                                   $\Delta$                          120                          106                        137                          78       0      2     10      1
Struct                             #                                9.1k                         8.6k                       7.4k                        6.1k    8.4k   8.0k   7.8k   8.4k
                                   $+$                              1.7k                         2.0k                        570                         694       4     83    257     68
                                   $-$                              1.0k                         1.9k                       1.6k                        3.1k       1    483    833    123
                                   $\Delta$                           81                          154                        116                         115       5     19     28     14
\shortstack{Tracept\\($\Delta=0$)} #                                 685                          690                        648                         559     752    747    739    752
                                   $+$                                45                           70                         25                          61       0      4     26      0
                                   $-$                               112                          132                        129                         254       0      9     39      0
Syscall                            #                                 291                          378                        347                         341     333    333    333    333
                                   $+$                                 2                           74                         23                          22       0      0      0      0
                                   $-$                                44                           29                          9                          14       0      0      0      0
Register                           $\Delta$                            1                            1                          1                           1       0      0      0      0